In [ ]:
import pandas as pd
import numpy as np
import MRT_tool
import matplotlib.pyplot as plt

### 用url取得資料(因為資料太大後來直接下載csv)

### 取得csv檔資料並轉成DataFrame

In [ ]:
MRT_URL = "臺北捷運每日分時各站OD流量統計資料_202401.csv"
time_list=["202401","202402"]
total_data=pd.DataFrame([])

for when in time_list:
    url_content=f"臺北捷運每日分時各站OD流量統計資料_{when}.csv"
    with open(url_content,encoding="utf-8") as f:
        df=pd.read_csv(f)

    # 清除人次為0
    df=df[(df["人次"]!=0)]
    
    # 將日期 的 type 轉換成 datetime
    df["日期"]=pd.to_datetime(df["日期"])
    df.index=df["日期"]
    del df["日期"]

    total_data=pd.concat([total_data,df])

total_data

### 捷運站篩選

In [ ]:
# 選擇捷運站
station_list=["圓山","東門","台北車站"]
total_station=[]

for station in station_list:
    print(f'正在取得{station}捷運站資料...')
    df_get=MRT_tool.df_get_station(total_data,station)
    print(f'已取得{station}捷運站資料')
    print(df_get)
    total_station.append(df_get)
print("資料擷取完成")

### 要分析哪站捷運站
- 用st做為所選捷運站的ID

In [ ]:
st=range(len(station_list))

In [ ]:
st_=total_station[st[0]]
st_

### 特定日期之進站出站量

In [ ]:
df=st_.loc["2024-02-01"]
df

In [ ]:
enter=df[df["進站"]==station_list[st[0]]]
leave=df[df["出站"]==station_list[st[0]]]
enter,leave

In [ ]:
x=sorted(list(set(df["時段"])))
y1=[enter[(enter["時段"]==i)]["人次"].sum() for i in x]
y2=[leave[(leave["時段"]==i)]["人次"].sum() for i in x]

In [ ]:
MRT_tool.plot_fig(f"{station_list[st[0]]}2024-02-01之進出站量","時間","總人數")

plt.xticks(np.linspace(0,23,24))

plt.plot(x,y1,"-o",x,y2,"-x")
plt.legend(["進站","出站"])

plt.savefig(f"{station_list[st[0]]}2024-02-01之進出站量.png")
plt.show()

### 一次分析所有挑選捷運站/特定日期下的出入站量

In [ ]:
time="2024-02-07"

MRT_tool.plot_fig(f"捷運站出入總人數{time} v.s. 時段(每小時)","時間","總人數")

plt.xticks(np.linspace(0,23,24))

plt_list=[]

for i in st:
    # print(total_station[st[i]])
    st_=total_station[st[i]]
    df=st_[st_.index==time]
    enter=df[df["進站"]==station_list[st[i]]]
    leave=df[df["出站"]==station_list[st[i]]]

    x=sorted(list(set(df["時段"])))
    y1=[enter[(enter["時段"]==i)]["人次"].sum() for i in x]
    y2=[leave[(leave["時段"]==i)]["人次"].sum() for i in x]

    
    plt.plot(x,y1,"-o",x,y2,"-x")
    plt_list.append(f"{station_list[st[i]]}進站")
    plt_list.append(f"{station_list[st[i]]}出站")

plt.legend(plt_list)

plt.savefig(f"捷運站出入總人數{time} v.s. 時段(每小時).png")
plt.show()


### 節日 vs 平日(二月/圓山)
- x軸: 每小時
- y軸: 每日平均人流出入量

In [ ]:
st_=total_station[st[0]]
st_["節日"]=False
st_

In [ ]:
st_[(st_.index=="2024-02-01") & (st_["時段"]==8)]["人次"].sum()

### 使用 holidays 抓出台灣的節日

In [ ]:
import holidays

In [ ]:
tw_holidays = holidays.TW()
tw_holidays

In [ ]:
# 測試
"2024-02-03" in tw_holidays

In [ ]:
st_["節日"]=[d in tw_holidays for d in st_.index]
st_[st_.index=="2024-02"]

In [ ]:
total_d=len(set(st_.index))
total_d

In [ ]:
tw_holidays

In [ ]:
df_hd=st_[st_["節日"]==True]
df_nd=st_[st_["節日"]==False]

x=sorted(list(set(st_["時段"])))
y1=[df_hd[(df_hd["時段"]==i)]["人次"].sum()/len(tw_holidays) for i in x]
y2=[df_nd[(df_nd["時段"]==i)]["人次"].sum()/(total_d-len(tw_holidays)) for i in x]

MRT_tool.plot_fig("節日 vs 平日(一月+二月/圓山)","時間","平均人數")

plt.xticks(np.linspace(0,23,24))

plt.plot(x,y1,"-o")
plt.plot(x,y2,"-x")

plt.legend(["節日","平日"])

plt.savefig(f"節日 vs 平日(一月+二月_圓山).png")
plt.show()

### 下雨是否影響捷運人潮?
- 雨天 vs 無雨

In [ ]:
df_rain=pd.read_csv("Rain_2024-02.csv",encoding="utf-8")

df_rain

In [ ]:
df_rain.index=pd.to_datetime(df_rain["Date"])
del df_rain["Date"]

In [ ]:
rain_dates=df_rain[df_rain["Rain"]==True]
rain_dates.index

In [ ]:
st_Feb=st_["2024-02":"2024-02"]
st_Feb

In [ ]:
for d in st_Feb.index:
    if d in df_rain.index:
        print(df_rain.loc[d,]["Rain"])

In [ ]:
st_Feb["下雨"]=[df_rain.loc[d,]["Rain"] for d in st_Feb.index]
st_Feb.loc["2024-02-01",]

In [ ]:
df_rd=st_Feb[st_Feb["下雨"]==True]
df_sd=st_Feb[st_Feb["下雨"]==False]

x=sorted(list(set(st_Feb["時段"])))
y1=[df_rd[(df_rd["時段"]==i)]["人次"].sum()/len(rain_dates) for i in x]
y2=[df_sd[(df_sd["時段"]==i)]["人次"].sum()/(len(df_rain)-len(rain_dates)) for i in x]

MRT_tool.plot_fig("雨天 vs 晴天(二月/圓山)","時間","平均人數")

plt.xticks(np.linspace(0,23,24))

plt.plot(x,y1,"-o")
plt.plot(x,y2,"-x")

plt.legend(["雨天","晴天"])

plt.savefig("雨天 vs 晴天(二月_圓山).png")
plt.show()

In [ ]:
df_rain=pd.read_csv("Rain_2024-01.csv",encoding="utf-8")

df_rain.index=pd.to_datetime(df_rain["Date"])

del df_rain["Date"]

rain_dates=df_rain[df_rain["Rain"]==True]

st_Jan=st_[:"2024-01"]

st_Jan["下雨"]=[df_rain.loc[d,]["Rain"] for d in st_Jan.index]

# 製圖

df_rd=st_Jan[st_Jan["下雨"]==True]
df_sd=st_Jan[st_Jan["下雨"]==False]

x=sorted(list(set(st_Jan["時段"])))
y1=[df_rd[(df_rd["時段"]==i)]["人次"].sum()/len(rain_dates) for i in x]
y2=[df_sd[(df_sd["時段"]==i)]["人次"].sum()/(len(df_rain)-len(rain_dates)) for i in x]

MRT_tool.plot_fig("雨天 vs 晴天(一月/圓山)","時間","平均人數")

plt.xticks(np.linspace(0,23,24))

plt.plot(x,y1,"-o")
plt.plot(x,y2,"-x")

plt.legend(["雨天","晴天"])

plt.savefig("雨天 vs 晴天(一月_圓山).png")
plt.show()